!pip install requests pandas folium

# But et étapes de ce notebook

## But : Créer un pipeline permettant de relier un déchet ménager à un "trash score"

## Liens à explorer

### Europe 

### France 

### Mailles 

#### National

In [ ]:
https://www.sinoe.org/thematiques/consult/ss-theme/1

#### Regional

In [ ]:
https://www.sinoe.org/filtres/index/thematique#table-annuaire

#### Ville

#### Entreprise

#### Consommateur

# Filière de recyclage

In [3]:
import requests
import pandas as pd
import folium

In [2]:
# Create df from API call
def create_df(url):

    # Send a request to the API and store the response
    response = requests.get(url)
    
    if response.status_code == 200:
        # Convert JSON response to a Python dictionary
        data = response.json()

        # Extract records from the data and create a pandas DataFrame
        records = data["records"]
        df = pd.DataFrame([record["fields"] for record in records])
        return df
    
    else:
        return f"Error: {response.status_code}"
    

## Centres de recyclage 

In [3]:
df_centres_recyclage = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=dechetteries-en-temps-reel&q=&facet=statut&facet=insee")


In [11]:
df_centres_recyclage.nom.value_counts()

CR Blanquefort               1
CR Merignac                  1
CR Bassens EXT               1
CR Eysines                   1
CR Le Taillan                1
CR Bordeaux Bastide          1
CR Saint-Medard-en-Jalles    1
CR Surcouf                   1
CR Ambes                     1
CR Pessac                    1
Name: nom, dtype: int64

In [42]:
df_centres_recyclage[:1]

,nom,ident,geo_point_2d,horaires_osm,geo_shape,mdate,cdate,adresse,statut,geom_o,gid,acceptes,insee,refuses
0,CR Blanquefort,CR Blanquefort,"[44.9274953, -0.6228383]","Mo 08:30-12:30,13:15-18:00; Tu 13:15-18:00; We...","{'coordinates': [-0.6228383, 44.9274953], 'typ...",2023-04-19,2023-04-19,"Avenue du 11 Novembre, 33290 Blanquefort",OUVERT,0,1,TOUT_VENANT_INCINERABLE;TOUT_VENANT_NON_INCINE...,33056,AMIANTE;BOUTEILLES_DE_GAZ;DECHETS_DELEMENTS_DA...


## Dechets et exutoires 

In [13]:
df_dechets = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=met_dechets_exutoires_conseils&q=&facet=nom_dechet&facet=famille_dechet&facet=type&rows=10000")


In [14]:
df_dechets

,id_dechet,famille_dechet,nom_dechet,type,label
0,8,Produits chimiques,Acide,Exutoire,Déchetterie
1,14,Plastique,Agitateur en plastique,Exutoire,Poubelle noire
2,27,Textiles,Alèse,Exutoire;Conseil;Exutoire,Borne textile;Don / Entraide / Revente;Déchett...
3,30,Vie domestique,Allume feu,Exutoire,Poubelle noire
4,32,Papier carton,Almanach,Exutoire;Conseil,Poubelle verte;Don / Entraide / Revente
...,...,...,...,...,...
1443,1379,Papier carton,Tube en carton,Exutoire;Conseil,Poubelle verte;Réutilisation
1444,1396,Transport,Véhicule,Conseil;Conseil;Exutoire,Don / Entraide / Revente;Réparation;Ferrailleu...
1445,1399,Sport,Vélo d'intérieur,Conseil;Conseil;Exutoire;Conseil,Réparation;Location;Déchetterie;Don / Entraide...
1446,1405,Vie domestique,Verre de table,Exutoire;Conseil,Poubelle noire;Don / Entraide / Revente


## Relations dechets et exutoires

In [15]:
df_relation_dechets_exut = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=met_dechets&q=&rows=10000")


In [17]:
df_relation_dechets_exut

,id_dechet,nom_dechet
0,3,Abricot
1,8,Acide
2,14,Agitateur en plastique
3,17,Agrume
4,19,Aiguille de couture
...,...,...
1443,1420,Visière de protection
1444,1422,Vitre en verre
1445,1428,Vtc
1446,1429,VTT


## Wording Dechets 

In [18]:
df_wording = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=met_exutoires_conseils&q=&facet&rows=10000")

In [20]:
df_wording

,id_exutoire,type,description,label
0,1,Exutoire,Ce déchet est recyclable. Attention : les déch...,Poubelle verte
1,2,Exutoire,Ce déchet n'est pour le moment pas recyclé.,Poubelle noire
2,3,Exutoire,Vous pouvez jeter ce déchet dans la borne à ve...,Borne à verre
3,4,Exutoire,vous pouvez apporter ce déchet en déchèteries.,Déchetterie
4,5,Exutoire,Afin de récupérer les textiles usagés ou que v...,Borne textile
5,6,Exutoire,30 % des ordures ménagères peuvent être recycl...,Compostage
6,9,Exutoire,"Même s'il y a un reste de médicament, ne serai...",Cyclamed
7,12,Exutoire,Vous pouvez les apporter dans des bornes de co...,Electronique - Grandes ou moyennes surfaces
8,15,Exutoire,Ce déchet doit être déposé chez un professionn...,Déchetterie professionnelle
9,16,Exutoire,Ce déchet est collecté dans des points de coll...,Points de collecte provisoires


## Bornes recyclage verre

In [21]:
df_bornes_verre = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=tal_bornes_collecte_verre&q=&rows=10000")    


In [22]:
df_bornes_verre

,geo_point_2d,geometry
0,"[44.844412, -0.797011]","{'coordinates': [-0.797011, 44.844412], 'type'..."
1,"[44.768781, -0.595476]","{'coordinates': [-0.595476, 44.768781], 'type'..."
2,"[44.763133, -0.626047]","{'coordinates': [-0.626047, 44.763133], 'type'..."
3,"[44.771803, -0.626471]","{'coordinates': [-0.626471, 44.771803], 'type'..."
4,"[44.878105, -0.681235]","{'coordinates': [-0.681235, 44.878105], 'type'..."
...,...,...
962,"[44.871333, -0.50784]","{'coordinates': [-0.50784, 44.871333], 'type':..."
963,"[44.800456, -0.635113]","{'coordinates': [-0.635113, 44.800456], 'type'..."
964,"[44.87128, -0.509267]","{'coordinates': [-0.509267, 44.87128], 'type':..."
965,"[44.865702, -0.595171]","{'coordinates': [-0.595171, 44.865702], 'type'..."


## Exploration des données OpenData France

### Tonnage de dechets par département

### Répartition du tonnage

In [227]:
df_tonnage_raw=pd.read_csv("/Users/WDescamps/Desktop/code_projects/side_projects/Trash_Score/France_waste_data/JD_ENQUETE_COLLECTE_DMA_TYPE_DECHET.csv", sep=';')
df_tonnage= df_tonnage_raw.copy()

In [228]:
df_tonnage.head(5)

,ANNEE,C_REGION,L_REGION,C_DEPT,N_DEPT,C_TYP_REG_DECHET,L_TYP_REG_DECHET,TONNAGE_DMA_T
0,2009,11,Ile-de-France,75,Paris,07A,Ordures ménagères résiduelles,"841716,00000"
1,2009,11,Ile-de-France,75,Paris,07B,Déblais et gravats,"5397,55682"
2,2009,11,Ile-de-France,75,Paris,07C,Encombrants,"97432,63023"
3,2009,11,Ile-de-France,75,Paris,07D,Déchets dangereux (y.c. DEEE),"456,08673"
4,2009,11,Ile-de-France,75,Paris,07E,Matériaux recyclables,"145215,47110"


In [229]:
# Quel département a transmis historiquement le plus de données?
df_tonnage.L_REGION.value_counts()

Occitanie                     545
Auvergne-Rhône-Alpes          504
Nouvelle-Aquitaine            502
Grand Est                     418
Bourgogne-Franche-Comté       336
Ile-de-France                 330
Centre-Val de Loire           251
Provence-Alpes-Côte d'Azur    250
Normandie                     210
Hauts-de-France               210
Pays de la Loire              208
Bretagne                      168
DROM-COM                      165
Corse                          84
Name: L_REGION, dtype: int64

#### Visualisation du tonnage Francais par département par année

In [230]:
fig = px.bar(
    df_tonnage,
    x="C_DEPT",
    y="TONNAGE_DMA_T",
    color="ANNEE",
    title="Tonnage of Different Waste Treatment Types",
    labels={"L_TYP_REG_DECHET": "Waste Treatment Type", "TONNAGE_DMA_T": "Tonnage"},
    hover_data=["C_DEPT", "L_REGION"],
    barmode="group",
)

fig.show()

### TOP 10 Waste type by Region 

In [231]:
df_tonnage.head(1)

,ANNEE,C_REGION,L_REGION,C_DEPT,N_DEPT,C_TYP_REG_DECHET,L_TYP_REG_DECHET,TONNAGE_DMA_T
0,2009,11,Ile-de-France,75,Paris,07A,Ordures ménagères résiduelles,"841716,00000"


In [232]:
df_tonnage["TONNAGE_DMA_T"] = [float(str(i).replace(",", "")) for i in df_tonnage["TONNAGE_DMA_T"]]

In [233]:
# Group by ANNEE and C_DEPT, and then sum the TONNAGE_T values
#grouped_df_tonnage = df_tonnage.groupby(["ANNEE", "L_REGION"]).agg({"TONNAGE_DMA_T": "sum"}).reset_index()

In [234]:
#grouped_df_tonnage.head(2)

In [219]:
# SELECT YEAR TO FILTER ON
filtered_grouped_df_tonnage = grouped_df_tonnage[grouped_df_tonnage["ANNEE"]==2019]

In [235]:
filtered_grouped_df_tonnage = filtered_grouped_df_tonnage.groupby(["L_REGION"]).agg({"TONNAGE_DMA_T": "sum"}).reset_index()


In [238]:
filtered_grouped_df_tonnage

,L_REGION,TONNAGE_DMA_T
0,Auvergne-Rhône-Alpes,4.351804e+11
1,Bourgogne-Franche-Comté,1.530057e+11
2,Bretagne,2.309325e+11
3,Centre-Val de Loire,1.473704e+11
4,Corse,2.559029e+10
5,DROM-COM,1.058847e+11
6,Grand Est,2.992135e+11
7,Hauts-de-France,3.613441e+11
8,Ile-de-France,5.722884e+11
9,Normandie,2.223013e+11


In [239]:
# Sort by TONNAGE_T in descending order and keep the top 10
top_10_df_tonnage = filtered_grouped_df_tonnage.sort_values(by="TONNAGE_DMA_T", ascending=False).head(10)


In [240]:
top_10_df_tonnage.sort_values(by="TONNAGE_DMA_T", ascending=False)

,L_REGION,TONNAGE_DMA_T
8,Ile-de-France,5.722884e+11
0,Auvergne-Rhône-Alpes,4.351804e+11
10,Nouvelle-Aquitaine,3.870800e+11
11,Occitanie,3.686067e+11
7,Hauts-de-France,3.613441e+11
13,Provence-Alpes-Côte d'Azur,3.597803e+11
6,Grand Est,2.992135e+11
2,Bretagne,2.309325e+11
9,Normandie,2.223013e+11
12,Pays de la Loire,2.215106e+11


In [241]:
fig = px.histogram(
    top_10_df_tonnage,
    x="TONNAGE_DMA_T",
    y="L_REGION",
    color="L_REGION",
    title="Top 10 C_DEPT with the Most Tonnage in 2019",
    labels={"TONNAGE_DMA_T": "Tonnage"},
    hover_data=["TONNAGE_DMA_T"],
)

fig.show()

#### Et si l'on regarde en divisant le nombre d'habitants?

In [244]:
# Add the approximate population data for each region in 2019
region_population = {
    "Ile-de-France": 12191246,
    "Auvergne-Rhône-Alpes": 7996854,
    "Nouvelle-Aquitaine": 5933660,
    "Occitanie": 5824717,
    "Hauts-de-France": 6014660,
    "Provence-Alpes-Côte d'Azur": 5030890,
    "Grand Est": 5522176,
    "Bretagne": 3325098,
    "Normandie": 3331215,
    "Pays de la Loire": 3739317,
}

# Map the population data to a new column "Population_2019"
top_10_df_tonnage["Population_2019"] = top_10_df_tonnage["L_REGION"].map(region_population)
top_10_df_tonnage["tonnage_par_habitant"]=top_10_df_tonnage["TONNAGE_DMA_T"]/top_10_df_tonnage["Population_2019"]


In [245]:
top_10_df_tonnage

,L_REGION,TONNAGE_DMA_T,Population_2019,tonnage_par_habitant
8,Ile-de-France,5.722884e+11,12191246,46942.569081
0,Auvergne-Rhône-Alpes,4.351804e+11,7996854,54418.948876
10,Nouvelle-Aquitaine,3.870800e+11,5933660,65234.616556
11,Occitanie,3.686067e+11,5824717,63283.200601
7,Hauts-de-France,3.613441e+11,6014660,60077.234428
13,Provence-Alpes-Côte d'Azur,3.597803e+11,5030890,71514.248895
6,Grand Est,2.992135e+11,5522176,54183.989215
2,Bretagne,2.309325e+11,3325098,69451.336874
9,Normandie,2.223013e+11,3331215,66732.800418
12,Pays de la Loire,2.215106e+11,3739317,59238.251230


In [247]:
fig = px.histogram(
    top_10_df_tonnage.sort_values(by="tonnage_par_habitant", ascending=False),
    x="tonnage_par_habitant",
    y="L_REGION",
    color="L_REGION",
    title="Top 10 C_DEPT with most tonnage/inhabitant in 2019",
    labels={"tonnage_par_habitant": "Tonnage/inhabitant"},
    hover_data=["tonnage_par_habitant"]
)

fig.show()

#### Focus sur la Gironde

In [248]:
# Pourquoi est ce que C_REGION == 75 pour la gironde???
df_tonnage_33=df_tonnage[df_tonnage["C_DEPT"] == "33"]
df_tonnage_33.head(2)

,ANNEE,C_REGION,L_REGION,C_DEPT,N_DEPT,C_TYP_REG_DECHET,L_TYP_REG_DECHET,TONNAGE_DMA_T
390,2009,75,Nouvelle-Aquitaine,33,Gironde,07A,Ordures ménagères résiduelles,4.009718e+10
391,2009,75,Nouvelle-Aquitaine,33,Gironde,07B,Déblais et gravats,7.712114e+09


In [55]:
df_tonnage_33.groupby(["ANNEE","L_TYP_REG_DECHET"])[["TONNAGE_DMA_T"]].agg(np.sum)

TONNAGE_DMA_T
ANNEE L_TYP_REG_DECHET                           
2009  Autres                           1574,19419
      Déblais et gravats              77121,13640
      Déchets dangereux (y.c. DEEE)    7083,60402
      Déchets verts et biodéchets    136668,99476
      Encombrants                     81656,82480
      Matériaux recyclables          169882,08917
      Ordures ménagères résiduelles  400971,77246
2011  Autres                           8291,49601
      Déblais et gravats              86013,10610
      Déchets dangereux (y.c. DEEE)    6015,04502
      Déchets verts et biodéchets    130563,89157
      Encombrants                     72705,79532
      Matériaux recyclables          187874,64151
      Ordures ménagères résiduelles  413195,97818
2013  Autres                          10117,13371
      Déblais et gravats              78642,65179
      Déchets dangereux (y.c. DEEE)    7082,07637
      Déchets verts et biodéchets    154924,14706
      Encombrants                     85444,81943
      Matériaux recyclables          189899,81697
      Ordures ménagères résiduelles  369169,39829
2015  Autres                           1194,88276
      Déblais et gravats              75663,88564
      Déchets dangereux (y.c. DEEE)    8488,67157
      Déchets verts et biodéchets    142997,37904
      Encombrants                     82960,69272
      Matériaux recyclables          197206,97668
      Ordures ménagères résiduelles  377077,96580
2017  Autres                            970,17355
      Déblais et gravats              83145,36112
      Déchets dangereux (y.c. DEEE)   10432,72516
      Déchets verts et biodéchets    153995,56072
      Encombrants                     95345,61609
      Matériaux recyclables          211502,06046
      Ordures ménagères résiduelles  393696,64030
2019  Autres                           1057,45613
      Déblais et gravats              80425,89388
      Déchets dangereux (y.c. DEEE)   11542,79530
      Déchets verts et biodéchets    161695,21724
      Encombrants                    105735,22006
      Matériaux recyclables          216172,87422
      Ordures ménagères résiduelles  394555,78653

### PLOT ! 

## Destination du tonnage

In [38]:
df_tonnage_destination_raw=pd.read_csv("/Users/WDescamps/Desktop/code_projects/side_projects/Trash_Score/France_waste_data/SINOE15_DestinationDecheterieParTraitement.csv", sep=',')
df_tonnage_destination= df_tonnage_destination_raw.copy()

In [39]:
df_tonnage_destination_33=df_tonnage_destination[df_tonnage_destination["C_DEPT"] == "33"]
df_tonnage_destination_33.head(2)

,ANNEE,C_REGION,L_REGION,C_DEPT,N_DEPT,C_TYP_REG_DECHET,L_TYP_REG_DECHET,C_TYP_REG_SERVICE,L_TYP_REG_SERVICE,TONNAGE_T
1366,2009,75,Nouvelle-Aquitaine,33,Gironde,02A,Déchets dangereux,02A,Valorisation matière,1155.55
1367,2009,75,Nouvelle-Aquitaine,33,Gironde,02A,Déchets dangereux,02B,Incinération avec récupération d'énergie,82.69


In [37]:
df_tonnage_destination_33

,ANNEE,C_REGION,L_REGION,C_DEPT,N_DEPT,C_TYP_REG_DECHET,L_TYP_REG_DECHET,C_TYP_REG_SERVICE,L_TYP_REG_SERVICE,TONNAGE_T
1366,2009,75,Nouvelle-Aquitaine,33,Gironde,02A,Déchets dangereux,02A,Valorisation matière,1155.55
1367,2009,75,Nouvelle-Aquitaine,33,Gironde,02A,Déchets dangereux,02B,Incinération avec récupération d'énergie,82.69
1368,2009,75,Nouvelle-Aquitaine,33,Gironde,02A,Déchets dangereux,02C,Incinération sans récupération d'énergie,5.10
1369,2009,75,Nouvelle-Aquitaine,33,Gironde,02A,Déchets dangereux,02D,Stockage,0.14
1370,2009,75,Nouvelle-Aquitaine,33,Gironde,02A,Déchets dangereux,02F,Stockage pour inertes,75.00
...,...,...,...,...,...,...,...,...,...,...
11334,2019,75,Nouvelle-Aquitaine,33,Gironde,02E,Déblais et gravats,02D,Stockage,2885.85
11335,2019,75,Nouvelle-Aquitaine,33,Gironde,02E,Déblais et gravats,02F,Stockage pour inertes,5507.00
11336,2019,75,Nouvelle-Aquitaine,33,Gironde,02Z,Autres déchets,02A,Valorisation matière,36.44
11337,2019,75,Nouvelle-Aquitaine,33,Gironde,02Z,Autres déchets,02E,Valorisation organique,7.31


### Tonnages annuels traités par type de filière recyclage

In [77]:
fig = px.bar(
    df_tonnage_destination_33,
    x="L_TYP_REG_SERVICE",
    y="TONNAGE_T",
    color="ANNEE",
    title="Tonnage of Different Waste Treatment Types",
    labels={"L_TYP_REG_SERVICE": "Waste Treatment Type", "TONNAGE_T": "Tonnage"},
    hover_data=["N_DEPT", "L_REGION"],
    barmode="group",
)

fig.show()

### Tonnages annuels par types de dechets 

In [67]:
fig = px.bar(
    df_tonnage_destination_33,
    x="L_TYP_REG_DECHET",
    y="TONNAGE_T",
    color="ANNEE",
    title="Tonnage of Types of Trash per Year",
    labels={"L_TYP_REG_SERVICE": "Waste Treatment Type", "TONNAGE_T": "Tonnage"},
    hover_data=["N_DEPT", "L_REGION"],
    barmode="group",
)

fig.show()

In [61]:
# Ces données ne sont pas assez précise, la maille C_TYP_REG_DECHET est la version aggrégée/regroupée
# de la colonne C_TYP_DECHET

## Lien entre C_TYP_DECHETS & C_TYP_REG_DECHETS

In [45]:
df_regroupement_dechets_raw= pd.read_csv("/Users/WDescamps/Desktop/code_projects/side_projects/Trash_Score/France_waste_data/SINOE42_NomenclatureRegroupementDechet.csv", sep=',')
df_regroupement_dechets=df_regroupement_dechets_raw.copy()
df_regroupement_dechets.head(2)

,C_TYP_AGR_DECHET,L_TYP_AGR_DECHET,C_TYP_REG_DECHET,L_TYP_REG_DECHET,C_TYP_DECHET,L_TYP_DECHET
0,1,Enquête collecte,01A,Ordures ménagères résiduelles,11.11,Ordures ménagères résiduelles
1,1,Enquête collecte,01B,Encombrants,07.31,Pneumatiques hors d'usage


In [49]:
import numpy as np

In [58]:
df_typ_reg_dechet=df_regroupement_dechets.groupby(["C_TYP_REG_DECHET", "L_TYP_REG_DECHET", "C_TYP_DECHET", "L_TYP_DECHET"])[["C_TYP_REG_DECHET"]].count()


In [60]:
df_typ_reg_dechet.tail(50)

C_TYP_REG_DECHET
C_TYP_REG_DECHET L_TYP_REG_DECHET C_TYP_DECHET L_TYP_DECHET                                                        
10Z              Autres déchets   10.12        Déchets de la préparation des produits végétaux                    1
                                  10.13        Corps gras                                                         1
                                  10.2         Déchets de produits alimentaires                                   1
                                  11           Déchets courants mélangés                                          1
                                  11.1         Déchets ménagers et assimilés                                      1
                                  11.11        Ordures ménagères résiduelles                                      1
                                  11.12        Déchets de voirie                                                  1
                                  11.13        Biodéchets                                                         1
                                  11.2         Déchets et matériaux en mélange                                    1
                                  11.21        Emballages sans les papiers graphiques                             1
                                  11.211       Emballages et papiers graphiques                                   1
                                  11.212       Emballages plastiques - metaux - briques                           1
                                  11.22        Déchets en mélange                                                 1
                                  12           Boues ordinaires                                                   1
                                  12.1         Boues d'épuration des eaux usées                                   1
                                  12.11        Boues d'épuration des eaux usées collectives                       1
                                  12.12        Boues ordinaires d'épuration des eaux usées                        1
                                  12.121       Digestat                                                           1
                                  12.2         Boues de traitement d'eaux potables                                1
                                  12.3         Boues de dragage                                                   1
                                  12.4         Matières de vidanges                                               1
                                  12.5         Boues de curage des égouts                                         1
                                  13.12        Déchets de revêtements routiers hydrocarbonés                      1
                                  13.13        Déchets de constructions et de démolition en mé...                 1
                                  13.15        Déchets de laines minérales                                        1
                                  13.151       Déchets de laines de roche                                         1
                                  13.152       Déchets de laines de verre                                         1
                                  13.2         Déchets de minéraux naturels                                       1
                                  13.21        DÉCHETS DE TERRES ET CAILLOUX                                      1
                                  13.22        Déchets de ballast non pollué                                      1
                                  13.3         Résidus d'opérations thermiques                                    1
                                  13.4         Déchets minéraux divers                                            1
                                  13.41        Déchets minéraux artificiels                                       1
                                  13.42

# Filière Produits

## Types de dechets Pères 

## Types de dechets 'fils'